### This Notebook is referred from here👇
 https://www.kaggle.com/goyalshalini93/car-price-prediction-linear-regression-rfe/notebook

### IMPORTING THE DATA

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
cars = pd.read_csv("../input/car-data/CarPrice_Assignment.csv")

### **Understanding the Data**

In [ ]:
cars.head()

In [ ]:
cars.columns

In [ ]:
cars.describe()

In [ ]:
cars.shape

In [ ]:
cars.info()

In [ ]:
cars["fueltype"].unique()

In [ ]:
Company_name = cars["CarName"].apply(lambda x: x.split(' ')[0])
print(Company_name.unique())

In [ ]:
cars.insert(3,"CompanyName", Company_name)
cars.drop(['CarName'], axis=1, inplace=True)
cars.head()

In [ ]:
cars.CompanyName.unique()

**correcting the incorrect name in the company name list**
*     maxda = mazda
*     Nissan = nissan
*     porcshce = porsche
*     toyota = toyouta
*     vokswagen = volkswagen = vw

In [ ]:
cars.CompanyName = cars.CompanyName.str.lower()

def replace_name(a,b):
    cars.CompanyName.replace(a,b, inplace=True)
    
replace_name('maxda', 'mazda')
replace_name('porcshce', 'porsche')
replace_name('toyouta', 'toyota')
replace_name('vw', 'volkswagen')
replace_name('vokswagen', 'volkswagen')

In [ ]:
cars.CompanyName.unique()

In [ ]:
cars.loc[cars.duplicated()]

In [ ]:
cars.columns

# Time to Visualize the data

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
plt.figure(figsize=(20,8))

plt.subplot(1,2,1)
plt.title("Car Price Plot")
sns.distplot(cars.price)

plt.subplot(1,2,2)
plt.title("Car Price Spread")
sns.boxplot(y=cars.price)

In [ ]:
print(cars.price.describe(percentiles = [0.25,0.50,0.75,0.85,0.90,1]))

<br>It can be observed that 85% of cars have price lesss than 18500.</br>
<br>and the remaing 15% has price between 18500 to 45400. </br>
<br> Also the mean and the median have a difference of nearly 3000</br>

### Ploting the Categorical data

In [ ]:
cars.info()

-  CompanyName
-  fueltype
-  carbody
-  doornumber
-  drivewheel
-  enginelocation
-  enginetype
-  cylindernumber
-  aspiration
-  fuelsystem

In [ ]:
plt.figure(figsize=(25,8))
# plt.xlabel(fontsize=18)
# plt.ylabel(fontsize=16)

plt.subplot(1,3,1)
plt1 = sns.countplot(data=cars, y='CompanyName')
# plt1 = cars.CompanyName.value_counts().plot(kind='bar')
plt.title("Companies Histogram")
plt1.set(ylabel="Company Name", xlabel="Frequency of company")


plt.subplot(1,3,2)
# plt2 = cars.fueltype.value_counts().plot(kind='bar')
plt2 = sns.countplot(data=cars, x='fueltype')
plt.title("Fuel Type Histogram")
plt2.set(xlabel="Fuel Type", ylabel="frequency")

plt.subplot(1,3,3)
# plt3 = cars.carbody.value_counts().plot(kind='bar')
plt3 = sns.countplot(data=cars, x='carbody')
plt.title("Car Body Histogram")
plt3.set(xlabel="Car body type", ylabel="Frequency")

plt.show()

* Toyota is dominating in all of this cars
* Gas type cars are more as compared to diesel
* And Sedan and Hatchback are thr favourite Car Body type among all 5

In [ ]:
plt.figure(figsize=(20,8))

plt.subplot(1,2,1)
plt.title("Symboling")
sns.countplot(cars.symboling, palette=("hls"))

plt.subplot(1,2,2)
plt.title("Symboling vs Price")
sns.boxplot(x=cars.symboling, y=cars.price, palette=("hls"))

## Conclusion
1. Symboling with 0 and 1 have high count, so can be said that they are sold at most
2. -1 symboling have are sold at high price

In [ ]:
plt.figure(figsize=(20,5))

plt.subplot(1,2,1)
plt.title("Cars Engine Type")
sns.countplot(x="enginetype", data=cars, palette=("Blues_d"))

plt.subplot(1,2,2)
plt.title("Cars Engine Type vs Price")
sns.boxplot(x="enginetype", y="price", data=cars, palette=("PuBuGn"))

plt.show()

df = pd.DataFrame(cars.groupby(['enginetype'])['price'].mean().sort_values(ascending=False))
df.plot.bar(figsize=(8,6))
plt.title('Engine Type vs Average Price')
plt.show()

### Inference:
1. dohcv have only 1 row but has a very high price
2. ohc type engine is used most and have less price
3. ohcv seems to have the highest price range

In [ ]:
plt.figure(figsize=(25,6))

df = pd.DataFrame(cars.groupby(['CompanyName'])['price'].mean().sort_values(ascending=False))
df.plot.bar()
plt.title("Company Name vs Average Price")
plt.show()

df = pd.DataFrame(cars.groupby(['fueltype'])['price'].mean().sort_values(ascending=False))
df.plot.bar()
plt.title("Fuel Type vs Average Price")
plt.show()

df = pd.DataFrame(cars.groupby(['carbody'])['price'].mean().sort_values(ascending=False))
df.plot.bar()
plt.title("Car Body vs Average Price")
plt.show()

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
plt.title('Door Number Histogram')
sns.countplot(x="doornumber", data=cars, palette=("plasma"))

plt.subplot(1,2,2)
plt.title('Door Number vs Price')
sns.boxplot(x=cars.doornumber, y=cars.price, palette=("plasma"))

plt.show()

plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
plt.title('Aspiration Histogram')
sns.countplot(cars.aspiration, palette=("plasma"))

plt.subplot(1,2,2)
plt.title('Aspiration vs Price')
sns.boxplot(x=cars.aspiration, y=cars.price, palette=("plasma"))

In [ ]:
def plot_count(x, fig):
    plt.subplot(4,2,fig)
    plt.title(x+' Histogram')
    sns.countplot(cars[x], palette=('magma'))
    plt.subplot(4,2,fig+1)
    plt.title(x+' vs Price')
    sns.boxplot(x=cars[x], y=cars['price'], palette=('magma'))
    

plt.figure(figsize=(15,20))

plot_count('enginelocation', 1)
plot_count('cylindernumber', 3)
plot_count('fuelsystem', 5)
plot_count('drivewheel', 7)

plt.tight_layout()

## Plotting the Numerical data

In [ ]:
cars.info()

In [ ]:
def scatter(x, fig):
    plt.subplot(5,2,fig)
    plt.scatter(cars[x], cars['price'])
    plt.title(x+' vs Price')
    plt.xlabel(x)
    plt.ylabel('Price')
    
plt.figure(figsize=(10,20))
    
scatter('carlength', 1)
scatter('carwidth', 2)
scatter('carheight', 3)
scatter('curbweight', 4)

plt.tight_layout()

1. Car height seems to be not related to price
2. carwidth, carlength, curbweight are correlated can can be seen in an increasing manner

In [ ]:
def pp(x):
    sns.pairplot(data=cars, x_vars=x, y_vars='price',size=3, kind='scatter')
    plt.show()
    
pp(['enginesize', 'boreratio', 'stroke'])
pp(['compressionratio', 'horsepower', 'peakrpm'])
pp(['wheelbase','citympg', 'highwaympg'])

1. enginesize, boreratio, horsepower,wheelbase seem to have positive correlation with price
2.  citympg, highwaymp are having negative corelation with price
3. stroke, compressionratio, peakrpm seem to be not having any correlation with price


In [ ]:
np.corrcoef(cars['carlength'], cars['carwidth'])[0,1]

## Deriving New Features

In [ ]:
#Fuel Economy
cars['fueleconomy'] = (0.55*cars['citympg']) + (0.45*cars['highwaympg'])

In [ ]:
#Binning the car Companies based on avg price of each company
cars['price'] = cars['price'].astype('int')
temp = cars.copy()
table = temp.groupby(['CompanyName'])['price'].mean()
temp = temp.merge(table.reset_index(), how='left', on='CompanyName')
bins = [0, 10000, 20000, 40000]
cars_bin=['Budget', 'Medium', 'Highend']
cars['carsrange'] = pd.cut(temp['price_y'], bins, right=False, labels=cars_bin)
cars.head()

In [ ]:
plt.figure(figsize=(8,6))
plt.title('Fuel Econmy vs Price')
sns.scatterplot(x=cars.fueleconomy, y=cars.price, hue=cars['drivewheel'])
plt.tight_layout()

In [ ]:
plt.figure(figsize=(25,6))
df = pd.DataFrame(cars.groupby(['fuelsystem', 'drivewheel','carsrange'])['price'].mean().unstack(fill_value=0))
df.plot.bar()
plt.title('Car Range vs Average Price')
plt.show()

In [ ]:
df.head()

### List of Significant Variables after visual analysis
- Fuel Type
- Car Type
- Engine Type
- Aspiration
- enginelocation
- drive wheel
- cylinfer no.
- Carlength
- Carwidth
- curbweight
- enginesize
- boreration
- horsepower
- fueleconomy
- Car Range

In [ ]:
cars_lr = cars[['price', 'fueltype', 'aspiration', 'carbody', 'drivewheel',
               'curbweight', 'enginetype', 'cylindernumber', 'enginesize',
               'boreratio','horsepower','fueleconomy', 'carlength',
                'carwidth','carsrange']]
cars_lr.head()

In [ ]:
sns.pairplot(cars_lr)
plt.show()

In [ ]:
def dummies(x,df):
    temp = pd.get_dummies(df[x], drop_first = True)
    df = pd.concat([df, temp], axis = 1)
    df.drop([x], axis = 1, inplace = True)
    return df
# Applying the function to the cars_lr

cars_lr = dummies('carsrange',cars_lr)
cars_lr = dummies('fueltype',cars_lr)
cars_lr = dummies('aspiration',cars_lr)
cars_lr = dummies('carbody',cars_lr)
cars_lr = dummies('drivewheel',cars_lr)
cars_lr = dummies('enginetype',cars_lr)
cars_lr = dummies('cylindernumber',cars_lr)


In [ ]:
cars_lr.head()

In [ ]:
cars_lr.shape

### Train and Test Split and feature scaling


In [ ]:
from sklearn.model_selection import train_test_split

np.random.seed(0)
df_train , df_test = train_test_split(cars_lr, train_size=0.7,
                                     test_size=0.3, random_state=100)


In [ ]:
cars_lr.info()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
num_vars = ['curbweight', 'enginesize', 'boreratio','horsepower',
           'fueleconomy', 'carlength','carwidth', 'price']

df_train[num_vars] = scaler.fit_transform(df_train[num_vars])

In [ ]:
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
#Correlation using Heatmap
plt.figure(figsize=(30,25))
sns.heatmap(df_train.corr(), annot=True, cmap='YlGnBu')
plt.show()

In [ ]:
#dividing data into X and y variable
y_train = df_train.pop('price')
X_train = df_train

## Model Building

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
# RFE == Recursive Feature Selection
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
lm = LinearRegression()
lm.fit(X_train, y_train)
rfe = RFE(lm, 10)
rfe = rfe.fit(X_train, y_train)

In [ ]:
list(zip(X_train.columns, rfe.support_, rfe.ranking_))

In [ ]:
X_train.columns[rfe.support_]

### Building Model using statsmodel, for the detailed statistics

In [ ]:
X_train_rfe = X_train[X_train.columns[rfe.support_]]
X_train_rfe.head()

In [ ]:
def build_model(X, y):
    X = sm.add_constant(X)
    lm = sm.OLS(y, X).fit()
    print(lm.summary())
    return X

In [ ]:
def checkVIF(X):
    vif = pd.DataFrame()
    vif['Features'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by="VIF", ascending=False)
    return vif

In [ ]:
X_train_new = build_model(X_train_rfe, y_train)

In [ ]:
X_train_new = X_train_rfe.drop(["twelve"], axis=1)

#### Model 2

In [ ]:
X_train_new = build_model(X_train_new, y_train)

In [ ]:
X_train_new = X_train_new.drop(["fueleconomy"], axis=1)

#### MODEL 3

In [ ]:
X_train_new = build_model(X_train_new, y_train)

In [ ]:
checkVIF(X_train_new)

In [ ]:
X_train_new = X_train_new.drop(["curbweight"], axis=1)

#### Model 4



In [ ]:
X_train_new = build_model(X_train_new, y_train)

In [ ]:
checkVIF(X_train_new)

droping sedan due to high VIF value

In [ ]:
X_train_new = X_train_new.drop(["sedan"], axis=1)

#### Model 5

In [ ]:
X_train_new = build_model(X_train_new, y_train)

In [ ]:
checkVIF(X_train_new)

Dropping wagon due to high value of p-value

In [ ]:
X_train_new = X_train_new.drop(['wagon'], axis=1)

### Model 6

In [ ]:
X_train_new = build_model(X_train_new, y_train)

In [ ]:
checkVIF(X_train_new)

## Residual Analysis of Model

In [ ]:
lm = sm.OLS(y_train, X_train_new).fit()
y_train_predict = lm.predict(X_train_new)

In [ ]:
fig = plt.figure()
sns.distplot((y_train - y_train_predict), bins=20)
fig.suptitle("Error terms", fontsize=20)
plt.xlabel("error", fontsize=18)

### Prediction and Evaluation

In [ ]:
num_vars = ['curbweight', 'enginesize', 'boreratio','horsepower',
           'fueleconomy', 'carlength','carwidth', 'price']
df_test[num_vars] = scaler.fit_transform(df_test[num_vars])

In [ ]:
#Dividing into X and Y
y_test = df_test.pop('price')
X_test = df_test

In [ ]:
X_test.columns

In [ ]:
#Now we are using the model to make prediction
X_train_new = X_train_new.drop('const', axis=1)
#Creating X_test_new by using same columns as X_train_new
X_test_new = X_test[X_train_new.columns]

#Adding a contant variable
X_test_new = sm.add_constant(X_test_new)

In [ ]:
#Make pedictions
y_pred = lm.predict(X_test_new)

Evaluation of test via comparison of y_pred and y_test

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

In [ ]:
#EVALUATION OF THE MODEL
# Plotting y_test and y_pred to understand the spread.
fig = plt.figure()
plt.scatter(y_test,y_pred)
fig.suptitle('y_test vs y_pred', fontsize=20)              # Plot heading 
plt.xlabel('y_test', fontsize=18)                          # X-label
plt.ylabel('y_pred', fontsize=16)

In [ ]:
print(lm.summary())